# Shapenet Rendering
- Blender script source: https://github.com/panmari/stanford-shapenet-renderer

In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import os
import sys
import json
import glob
import cv2
import imageio
import shutil

import subprocess
import numpy as np

from modules.ShapeNetHandler import ShapeNetHandler
import modules.utils

# Load ShapeNet Handler

In [3]:
shapenet_root = "/hdd/mliuzzolino/ShapeNet/data/ShapeNetCore.v2"
shapenet_handler = ShapeNetHandler(shapenet_root)

## Print available categories by name
- Can use name to condition random sampling

In [4]:
# shapenet_handler.print_categories()

## Randomly Sample a filepath

In [5]:
obj_filepath, synset_id, instance_id = shapenet_handler.sample_obj(category_name='car')

SynsetID: 02958343 -- InstanceID: 921f87f3d9a4cb8ddb37d391409760c4 -- Name: car,auto,automobile,machine,motorcar


## Output Path

In [6]:
_OUTPUT_ROOT = 'render_output'

# Render

In [65]:
_N_VIEW_ANGLES = 72
_N_MODELS_TO_RENDER = 20
_RANDOM_SCALING = True
_OVERWRITE = True
_RENDER_SCRIPT = "scripts/blender_render.py"
_TARGET_CATEGORY_LIST = ['airplane', 'car']

## Run Blender

In [ ]:
subprocess_outputs = []
try:
    for img_i in range(_N_MODELS_TO_RENDER):
        # Randomly select target category
        target_category = np.random.choice(_TARGET_CATEGORY_LIST)
        
        # Sample object filepath
        obj_filepath, synset_id, instance_id, obj_name = shapenet_handler.sample_obj(category_name=target_category)
        
        # Setup outpath
        output_root = os.path.join(_OUTPUT_ROOT, f'synsetID_{synset_id}', instance_id)
        
        # Randomly select obj scaling
        random_scale = np.random.uniform(0.75, 1.0) if _RANDOM_SCALING else 1.0
        
        if not os.path.exists(output_root) or _OVERWRITE:
            # Remove previous directory if overwrite
            if _OVERWRITE and os.path.exists(output_root):
                shutil.rmtree(output_root)
            
            # Run blender script
            cmd = f"blender --background --python {_RENDER_SCRIPT} -- "
            cmd += f"--output_folder {output_root} --views {_N_VIEW_ANGLES} "
            cmd += f"--scale {random_scale} "
            cmd += f"{obj_filepath}"
            out = subprocess.check_output(cmd.split(" "), shell=False)
            subprocess_outputs.append(out)
        else:
            print("Skipping.")
        
        # Stdout
        stdout_str = f"[{img_i+1}/{_N_MODELS_TO_RENDER}] -- "
        stdout_str += f"SynsetID: {synset_id} -- "
        stdout_str += f"InstanceID: {instance_id:32s} -- "
        stdoust_str += f"Name: {obj_name}"
        print(stdout_str)
        
except KeyboardInterrupt:
    try:
        # Try to cleanup directory with incomplete # renders
        if len(os.listdir(output_root)) != _N_VIEW_ANGLES:
            shutil.rmtree(output_root)
    except:
        pass
    print("\nEnding early.")

SynsetID: 02691156 -- InstanceID: 9fb0988ef701a11388170d426b6605b5 -- Name: airplane,aeroplane,plane
SynsetID: 02958343 -- InstanceID: 1641efa5c92514d86c4f4dbcda5f2fc0 -- Name: car,auto,automobile,machine,motorcar
SynsetID: 02691156 -- InstanceID: ed7e1a38fe33830b87697d3904b168b -- Name: airplane,aeroplane,plane
SynsetID: 02691156 -- InstanceID: 28d719b73b1fcd2f5de04aad18bd94c3 -- Name: airplane,aeroplane,plane
SynsetID: 02958343 -- InstanceID: a72134cd499fd1c4f79e091fa09130a -- Name: car,auto,automobile,machine,motorcar
SynsetID: 02958343 -- InstanceID: 71a1122469df0bb8cc8ad0deed05da5c -- Name: car,auto,automobile,machine,motorcar
SynsetID: 02691156 -- InstanceID: 12e127c0416f94ca4c3a35cee92bb95b -- Name: airplane,aeroplane,plane
SynsetID: 02958343 -- InstanceID: 65d6433043c40046b82c0841410a924f -- Name: car,auto,automobile,machine,motorcar
SynsetID: 02958343 -- InstanceID: bc7ca9eec356ce21ad32396cbf7f43dc -- Name: car,auto,automobile,machine,motorcar
SynsetID: 02958343 -- InstanceID:

## Background Images

In [ ]:
background_imgs_root = '/hdd/mliuzzolino/Places2/places365_standard/train'
background_img_paths = glob.glob(f'{background_imgs_root}/*/*')

## Load Images and Animate

In [ ]:
def add_background(imgs, background_img):
    RGB_img, mask = imgs
    RGB_img[np.where(~mask)] = background_img[np.where(~mask)]
    return RGB_img, mask

In [ ]:
_LOAD_LIMIT = 15

limit_reached = False

all_imgs = []
rendered_synset_ids = glob.glob(f"{_OUTPUT_ROOT}/*")
np.random.shuffle(rendered_synset_ids)
for rendered_synset_id in rendered_synset_ids:
    rendered_instance_ids = glob.glob(f"{rendered_synset_id}/*")
    for rendered_instance_id in rendered_instance_ids:
        sys.stdout.write(f"\rProcessed model {len(all_imgs)+1}/{_LOAD_LIMIT}...")
        sys.stdout.flush()
        rendered_img_paths = np.sort(glob.glob(f"{rendered_instance_id}/*"))
        
        rendered_imgs = [modules.utils.read_image(img_path, same_size=True) 
                             for img_path in rendered_img_paths]
        
        random_background_path = np.random.choice(background_img_paths)
        background_img = imageio.imread(random_background_path)
        shape = rendered_imgs[0][0].shape[:2][::-1]
        background_img = cv2.resize(background_img, shape)
        rendered_imgs = [add_background(img, background_img) for img in rendered_imgs]
        if len(rendered_imgs) != _N_VIEW_ANGLES:
            print(f"\t**Error: Insufficient angles! Actual: {len(rendered_imgs)} - Expected: {_N_VIEW_ANGLES}")
            continue
            
        combined_imgs = modules.utils.combine_imgs(rendered_imgs)
        all_imgs.append(combined_imgs)
        
        if len(all_imgs) >= _LOAD_LIMIT:
            print("Breaking..")
            limit_reached = True
            break
    
    if limit_reached:
        break

In [ ]:
# Stack all images
stacked_imgs = modules.utils.stack_all_imgs(all_imgs, nrow=3)

In [ ]:
modules.utils.toAnimation(stacked_imgs, figsize=(12,12), interval=100, savepath='test.mp4', fps=15)